# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.
There are two options for communcating with Julia: `PyCall.jl/pyjulia` and `PythonCall.jl/juliacall`.
Here we use the second by passing `calljulia="juliacall"` when initializing.

In [1]:
import qiskit_alt
qiskit_alt.project.ensure_init(calljulia="juliacall")

We assume that no one is familiar with Julia, much less with `juliacall`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `juliacall` works, we create an `Array` of `Float64` zeros on the Julia side. On the Python side, they are dispalyed as they would in Julia.

In [2]:
julia = qiskit_alt.project.julia
Main = julia.Main
julia.Main.zeros(3)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

However, we can see that the Julia `Vector` is in fact wrapped in a Python class.

In [3]:
type(julia.Main.zeros(3))

juliacall.VectorValue

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into much of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [4]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [5]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [6]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [7]:
import qiskit_alt.electronic_structure
fermi_op = qiskit_alt.electronic_structure.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.electronic_structure.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

Note that the constant term differs. The qiskit-nature version ignores the nuclear-repulsion term. I need to open an issue about whether and how to handle it.

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [8]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 3.98 ms
geometry=h2_geometry, basis='631g' 47.24 ms
geometry=h2_geometry, basis='631++g' 253.31 ms
geometry=h2o_geometry, basis='sto3g' 294.54 ms
geometry=h2o_geometry, basis='631g' 3758.89 ms


[3.978773101698607,
 47.24139650352299,
 253.30708539113402,
 294.5441839983687,
 3758.8944919407368]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [9]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.18 ms
geometry=h2_geometry, basis='631g' 2.22 ms
geometry=h2_geometry, basis='631++g' 19.91 ms
geometry=h2o_geometry, basis='sto3g' 20.40 ms
geometry=h2o_geometry, basis='631g' 540.90 ms


[0.18255289178341627,
 2.2248483961448073,
 19.910018099471927,
 20.400322100613266,
 540.8974146004766]

We compare the relative performance.

In [10]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[21.79517981242986,
 21.233535096315936,
 12.72259443088364,
 14.438212423592773,
 6.949366719966985]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [11]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 62.80 ms
geometry=h2_geometry, basis='631g' 103.84 ms
geometry=h2_geometry, basis='631++g' 293.04 ms
geometry=h2o_geometry, basis='sto3g' 453.94 ms
geometry=h2o_geometry, basis='631g' 26602.88 ms


[62.802629196085036,
 103.84225139860064,
 293.0431596003473,
 453.9409728022292,
 26602.879529003985]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [12]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 60.73 ms
geometry=h2_geometry, basis='631g' 64.57 ms
geometry=h2_geometry, basis='631++g' 66.66 ms
geometry=h2o_geometry, basis='sto3g' 78.42 ms
geometry=h2o_geometry, basis='631g' 189.03 ms


[60.727139201480895,
 64.56560799852014,
 66.66306089609861,
 78.41791190439835,
 189.02676939032972]

We compare the relative performance.

In [13]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.034177305598376,
 1.6083214364059073,
 4.39588515230475,
 5.788740885572705,
 140.73604291501445]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.com/Qiskit-Extensions/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.com/Qiskit-Extensions/ElectronicStructure.jl) provides an interface to electronic structure packages.

* [`ElectronicStructurePySCF.jl`](https://github.com/Qiskit-Extensions/ElectronicStructurePySCF.jl) provides an interface to `pyscf`

* [`QiskitQuantumInfo.jl`](https://github.com/Qiskit-Extensions/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* [Issue](https://github.com/Qiskit-Extensions/QuantumOps.jl/issues/17) for improving performance of Jordan-Wigner in `QuantumOps`.
    * Precompute one and two-body terms
    * Use @threads


# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [14]:
from qiskit_alt.pauli_operators import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
m = Main.convert(Main.Matrix, m) # Convert PythonCall.PyArray to a native Julia type
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.37277278+0.j        ,  0.48141813+0.j        ,  0.        +0.0991416j ,
  0.05361269+0.j        ,  0.27520131+0.j        ,  0.42186284+0.j        ,
  0.        -0.04982752j, -0.0230298 +0.j        ,  0.        -0.01717269j,
  0.        -0.17941261j,  0.06135846+0.j        ,  0.        -0.00710431j,
 -0.11456777+0.j        , -0.03505587+0.j        ,  0.        +0.06769027j,
  0.16526017+0.j        ,  0.50582191+0.j        ,  0.657631  +0.j        ,
  0.        -0.09945913j, -0.08430098+0.j        ,  0.47055

When using `pyjulia`, the `numpy` matrix is automatically converted to a Julia `Matrix`.
But, here, we are using `juliacall`. In this case the `numpy` matrix is wrapped in `PythonCall.PyArray`, a subtype of `AbstractMatrix` that is particular to `juliacall`. It supports iteration in Julia, but the iterator calls python. The performance in this case is much worse. For this reason, we convert it to a matrix of Julia floats via
`m = Main.convert(Main.Matrix, m)`

We run benchmarks of conversion of matrices to the Pauli basis.

In [15]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.24 ms
nqubits=3, 5.00 ms
nqubits=4, 24.25 ms
nqubits=5, 139.00 ms
nqubits=6, 1655.33 ms
nqubits=7, 13171.83 ms
nqubits=8, 148521.77 ms


In [16]:
%run ../bench/from_matrix_alt.py

nqubits=2, 0.53 ms
nqubits=3, 0.59 ms
nqubits=4, 8.65 ms
nqubits=5, 24.48 ms
nqubits=6, 25.11 ms
nqubits=7, 101.66 ms
nqubits=8, 543.59 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [17]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[2.35246695659807,
 8.497287538045084,
 2.802590648569309,
 5.67711136738918,
 65.9263227910199,
 129.56868853648788,
 273.2230323124041]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [18]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.36637530429288745 ms
k=10, n=100, 1.8803650047630072 ms
k=10, n=1000, 16.845163953257725 ms
k=10, n=5000, 83.35850099683739 ms
k=10, n=10000, 169.82630197890103 ms
k=10, n=100000, 1710.816810023971 ms
k=100, n=10, 0.39985135081224144 ms
k=100, n=100, 2.0084912015590817 ms
k=100, n=1000, 18.190109147690237 ms
k=100, n=5000, 91.80668409680948 ms
k=100, n=10000, 185.64801895990968 ms
k=100, n=100000, 1862.4763570260257 ms


Now, using `qiskit_alt`

In [19]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.08800504729151726 ms
k=10, n=100, 0.11579025303944945 ms
k=10, n=1000, 0.528139848029241 ms
k=10, n=5000, 2.7049661963246763 ms
k=10, n=10000, 6.397994997678325 ms
k=10, n=100000, 76.1052493005991 ms
k=100, n=10, 0.10077175102196634 ms
k=100, n=100, 0.21615325240418315 ms
k=100, n=1000, 1.4627085998654366 ms
k=100, n=5000, 7.4819949513766915 ms
k=100, n=10000, 15.525550750317052 ms
k=100, n=100000, 202.44699189788662 ms


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [20]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[4.1631169525909915,
 16.239406646104932,
 31.89527170902862,
 30.816836495073115,
 26.543675329619173,
 22.47961639632266,
 3.967891266720982,
 9.291977702021438,
 12.43590770531031,
 12.27034830863084,
 11.95758024597733,
 9.199822331593104]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia takes all the time.

We see this in the following.

These magics are not defined for `juliacall`, so we skip them.

In [26]:
import qiskit.tools.jupyter
d = qiskit.__qiskit_version__._version_dict
d['qiskit_alt'] = qiskit_alt.__version__
%qiskit_version_table

/home/lapeyre/julia_qiskit_repos/qiskit_alt/venv/lib/python3.9/site-packages/qiskit/tools/jupyter/__init__.py:139: RuntimeWarning: matplotlib can't be found, ensure you have matplotlib and other visualization dependencies installed. You can run '!pip install qiskit-terra[visualization]' to install it from jupyter
  warnings.warn(


In [21]:
# %load_ext julia.magic

Generate `1000` ten-qubit Pauli strings.

In [22]:
# %julia using Random: randstring
#%julia pauli_strings = [randstring("IXYZ", 10) for _ in 1:1000]
# None;

Benchmark converting these to a `QuantumOps.PauliSum`. Note that as the sums are always sorted.

In [23]:
# %julia import Pkg; Pkg.add("BenchmarkTools")
#%julia using BenchmarkTools: @btime
#%julia @btime QuantumOps.PauliSum($pauli_strings)
#None;

Check that we are actually getting the right result.

In [24]:
#%julia pauli_sum = QuantumOps.PauliSum(pauli_strings);
#%julia println(length(pauli_sum))
#%julia println(pauli_sum[1])

In [25]:
#6.9 * 2.29 / .343  # Ratio of time to construct PauliSum via qiskit_alt to time in pure Julia

So the pure Julia code is `46` times faster than the qiskit.quantum_info.
**But, the `qiskit.quantum_info` is also extracting a possible phase !**